# US fires map (1000+ acres)
Messy notebook that stitches together the `us_fires` CSV shards and makes a map with a year slider.

In [ ]:
import glob
import pandas as pd
import plotly.express as px

data_dir = '2018-07-wildfire-trends/data/us_fires'
files = sorted(glob.glob(f'{data_dir}/us_fires_*.csv'))
files


In [ ]:
# Load and concatenate
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append(df)
fires = pd.concat(dfs, ignore_index=True)
fires.shape


In [ ]:
# Basic cleanup: keep only 1000+ acre fires and contiguous US
fires['fire_size'] = pd.to_numeric(fires['fire_size'], errors='coerce')
fires['fire_year'] = pd.to_numeric(fires['fire_year'], errors='coerce')
fires['latitude'] = pd.to_numeric(fires['latitude'], errors='coerce')
fires['longitude'] = pd.to_numeric(fires['longitude'], errors='coerce')

contig_exclude = {'AK', 'HI', 'PR'}
filtered = fires[
    (fires['fire_size'] >= 1000) &
    (fires['state'].notna()) &
    (~fires['state'].isin(contig_exclude))
].copy()

# Drop rows missing coords/year
filtered = filtered.dropna(subset=['latitude', 'longitude', 'fire_year'])
filtered[['fire_year','fire_size']].describe()


In [ ]:
# Map with size encoded by fire_size, and a year slider
# Fix colorbar range across all years
min_size = filtered['fire_size'].min()
max_size = filtered['fire_size'].max()

fig = px.scatter_geo(
    filtered,
    lat='latitude',
    lon='longitude',
    size='fire_size',
    color='fire_size',
    hover_name='fire_name',
    animation_frame='fire_year',
    scope='usa',
    size_max=20,
    range_color=(min_size, max_size),
    title='US Fires (1000+ acres) with Year Slider'
)
fig.update_layout(geo=dict(scope='usa'))
fig.show()
